# Rate and temperature depedent 1D SLIDE model

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

### Model implementation (single material point)

In [ ]:
def get_bond_slip(s_arr, dt, tau_pi_bar, K, gamma, E_b, S, c, r, m, sigma_n, eta, Cv, S0, T_bar):

    # arrays to store the values
    tau_arr = np.zeros_like(s_arr)
    w_arr = np.zeros_like(s_arr)
    s_pi_arr = np.zeros_like(s_arr)
    s_vis_arr = np.zeros_like(s_arr)
    f = np.zeros_like(s_arr)
    
    t_arr = np.zeros_like(s_arr)
    
    # initial state
    tau_i = 0
    alpha_i = 0.
    s_pi_i = 0
    z_i = 0.
    w_i = 0.
    s_vis_i = 0.
    t_i = 0.


    for i in range(1, len(s_arr)):
        
        s_i = s_arr[i]
        tau_trial = (1 - w_i) * E_b * (s_i - s_pi_i)
        tau_i = tau_trial
        
        z_n = z_i
        alpha_n = alpha_i

        # Threshold
        f_pi_i = np.fabs(tau_trial/(1. - w_i) - gamma * alpha_i) - tau_pi_bar - K * z_i + m * sigma_n

        if f_pi_i > 1e-6:
            # Return mapping
            delta_lamda = f_pi_i / (E_b / (1. - w_i) + gamma + K)
            # update all the state variables
            s_pi_i +=  delta_lamda * np.sign(tau_trial/(1. - w_i) - gamma * alpha_i) / (1 - w_i)
            Y_i = 0.5 * E_b * (s_i - s_pi_i) ** 2
            w_i += ((1. - w_i) ** c) * (delta_lamda * (Y_i / S) ** r) * (1 + T_bar * 1/(1+np.exp(-t_i)))
            alpha_i += delta_lamda * np.sign(tau_trial/(1. - w_i) - gamma * alpha_i)
            z_i +=  delta_lamda
            tau_i = E_b * (1. - w_i) * (s_i - s_pi_i)
            
            
           # perform viscoplastic regularization
            relax_t = eta / (E_b + K + gamma)
             
            tau_i = (tau_trial + (dt/relax_t) * tau_i)/(1. + dt/relax_t)  
            z_i = (z_n + (dt/relax_t) * z_i)/(1. + dt/relax_t) 
            alpha_i = (alpha_n + (dt/relax_t) * alpha_i)/(1. + dt/relax_t) 
            s_vis_i = s_i - tau_i/E_b
            
            
            # Evaluate temperature
            t_i =  t_i + (np.abs(s_vis_i)*(1./S0))
            #t_i =  t_i + (z_i*(1./S0))


        tau_arr[i] = tau_i
        w_arr[i] = w_i
        s_pi_arr[i] = s_pi_i
        s_vis_arr[i] = s_vis_i
        
        t_arr[i] = t_i

    return s_arr, tau_arr, w_arr, s_pi_arr, s_vis_arr, t_arr


### Apply loading

In [ ]:
# Define the loading values

Number_of_cycles = 25
maximum_slip = 0.035

number_of_increments_1 = 50

# Construction of the loading history
s_levels = np.linspace(0, maximum_slip, 2 * Number_of_cycles )
s_levels.reshape(-1, 2)[:, 0] = maximum_slip
s_levels[0] = 0
s_levels.reshape(-1, 2)[:, 1] = 0.3 * maximum_slip
s_history = s_levels.flatten()


# slip array as input
s_arr_1 = np.hstack([np.linspace(s_history[i], s_history[i + 1], number_of_increments_1)
                     for i in range(len(s_levels) - 1)])

# material parameters
tau_pi_bar=1.5
K=0
gamma=10000
E_b=100
S=0.0005
c=1.2
r=1.0
m=0.0
sigma_n=0
eta = 10000


Cv = 0.0 # 1.0
#S0 = 0.0025
T_bar = 0.0


dt = 1./number_of_increments_1
s_arr_1 = s_arr_1[:100]
time_1 = np.linspace(0,1, len(s_arr_1))

# Runing the simulation
s_arr_1,  tau_arr_1, w_arr_1, s_pi_arr_1, s_vis_arr_1, t_arr_1 = get_bond_slip(
    s_arr_1, dt,tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S,
    c=c, r=r, m=m, sigma_n=sigma_n, eta= eta, Cv=Cv, S0=1.0e+5, T_bar = T_bar)

s_arr_1,  tau_arr_2, w_arr_2, s_pi_arr_2, s_vis_arr_2, t_arr_2 = get_bond_slip(
    s_arr_1, dt, tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S,
    c=c, r=r, m=m, sigma_n=sigma_n, eta= eta, Cv=Cv, S0=1.0e+5, T_bar = T_bar)
dt

### Plotting

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ((ax_1,  ax_2), (ax_3,  ax_4)) = plt.subplots(2,2, tight_layout=True, figsize=(9.0,7))
fig.canvas.header_visible=False


ax1 = plt.subplot(221)
ax1.plot(s_arr_1, tau_arr_1, '-k', linewidth=3, label="S0 = 0.25")
ax1.plot(s_arr_1, tau_arr_2, '-r', linewidth=1, label="S0 = 1.0")
ax1.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax1.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('stress-slip')
plt.xlabel('Slip (mm)')
plt.ylabel('Stress (MPa)')
plt.legend(loc=4)


ax2 = plt.subplot(222)
ax2.plot(time_1, s_vis_arr_1, '-k', linewidth=3, label="S0 = 0.25")
ax2.plot(time_1, s_vis_arr_2, '-r', linewidth=1, label="S0 = 1.0")
ax2.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax2.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Evolution of viscoplastic strain')
plt.xlabel('Slip(mm)')
plt.ylabel('viscoplastic strain')
plt.legend()
 

ax3 = plt.subplot(223)
ax3.plot(time_1, w_arr_1, 'k', linewidth=3, label="S0 = 0.25")
ax3.plot(time_1, w_arr_2, 'r', linewidth=1, label="S0 = 1.0")
ax3.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax3.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Damage evolution')
plt.xlabel('Slip(mm)')
plt.ylabel('damage')
plt.legend()



ax4 = plt.subplot(224)
ax4.plot(time_1, t_arr_1, 'k', linewidth=2, label="S0 = 0.25")
ax4.plot(time_1, t_arr_2, 'r', linewidth=2, label="S0 = 1.0")
ax4.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax4.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Temperature evolution')
plt.xlabel('Slip(mm)')
plt.ylabel('Temperature ')
plt.legend()

#plt.show()

In [ ]:
# Define the loading values

Number_of_cycles = 25
maximum_slip = 0.035

number_of_increments_1 = 50

# Construction of the loading history
s_levels = np.linspace(0, maximum_slip, 2 * Number_of_cycles )
s_levels.reshape(-1, 2)[:, 0] = maximum_slip
s_levels[0] = 0
s_levels.reshape(-1, 2)[:, 1] = 0.3 * maximum_slip
s_history = s_levels.flatten()


# slip array as input
s_arr_3 = np.hstack([np.linspace(s_history[i], s_history[i + 1], number_of_increments_1)
                     for i in range(len(s_levels) - 1)])

time_3 = np.linspace(0,1, len(s_arr_1))


# material parameters
tau_pi_bar=1.5
K=0
gamma=10000
E_b=100
S=0.0005
c=1.2
r=1.0
m=0.0
sigma_n=0
eta = 10000


Cv =1.0
S0= 0.25
#T_bar = 0.0


dt = 1./number_of_increments_1


# Runing the simulation
s_arr_3,  tau_arr_3, w_arr_3, s_pi_arr_3, s_vis_arr_3, t_arr_3 = get_bond_slip(
    s_arr_3, dt,tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S,
    c=c, r=r, m=m, sigma_n=sigma_n, eta= eta, Cv=Cv, S0=S0, T_bar = 0)

s_arr_3,  tau_arr_4, w_arr_4, s_pi_arr_4, s_vis_arr_4, t_arr_4 = get_bond_slip(
    s_arr_3, dt, tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S,
    c=c, r=r, m=m, sigma_n=sigma_n, eta= eta, Cv=Cv, S0=S0, T_bar = 1)

In [ ]:
import matplotlib.pylab as plt
fig, ((ax_1,  ax_2), (ax_3,  ax_4)) = plt.subplots(2,2, tight_layout=True, figsize=(9.0,7))
fig.canvas.header_visible=False


ax1 = plt.subplot(221)
ax1.plot(s_arr_3, tau_arr_3, '-k', linewidth=2, label="T_bar = 0")
ax1.plot(s_arr_3, tau_arr_4, '-g', linewidth=2, label="T_bar = 1")
ax1.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax1.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('stress-slip')
plt.xlabel('Slip (mm)')
plt.ylabel('Stress (MPa)')
plt.legend(loc=4)


ax2 = plt.subplot(222)
ax2.plot(time_3, s_vis_arr_3, '-k', linewidth=2, label="T_bar = 0")
ax2.plot(time_3, s_vis_arr_4, '-g', linewidth=2, label="T_bar = 1")
ax2.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax2.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Evolution of viscoplastic strain')
plt.xlabel('Slip(mm)')
plt.ylabel('viscoplastic strain')
plt.legend()
 

ax3 = plt.subplot(223)
ax3.plot(time_3, w_arr_3, 'k', linewidth=2, label="T_bar = 0")
ax3.plot(time_3, w_arr_4, 'g', linewidth=2, label="T_bar = 1")
ax3.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax3.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Damage evolution')
plt.xlabel('Slip(mm)')
plt.ylabel('damage')
plt.legend()



ax4 = plt.subplot(224)
ax4.plot(time_3, t_arr_3, 'k', linewidth=2, label="T_bar = 0")
ax4.plot(time_3, t_arr_4, 'g', linewidth=2, label="T_bar = 1")
ax4.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax4.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Temperature evolution')
plt.xlabel('Slip(mm)')
plt.ylabel('Temperature ')
plt.legend()

plt.show()

## Verification for a single step calculation

In [ ]:
# slip array as input
number_of_increments_1 = 20

_u_Tx = 1/50*1.1 * 2
s_arr_1 = np.linspace(0.0, _u_Tx, number_of_increments_1)

time_1 = np.linspace(0,1, len(s_arr_1))

# material parameters
tau_pi_bar=1
K=1
gamma=1
E_b=50
S=1e-4
c=1.2
r=1.0
m=0.0
sigma_n=0
tau_relax = 0.01
eta = E_b * tau_relax

#  E_T_=50, gamma_T_=1, K_T_=1, S_T_=1e-4, c_T_=1.2, r_T_=1.0, f_s_=1, 

Cv = 0.0 # 1.0
#S0 = 0.0025
T_bar = 0.0


dt = 1./number_of_increments_1

In [ ]:
s_arr_1,  tau_arr_1, w_arr_1, s_pi_arr_1, s_vis_arr_1, t_arr_1 = get_bond_slip(
    s_arr_1, dt,tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S,
    c=c, r=r, m=m, sigma_n=sigma_n, eta= eta, Cv=Cv, S0=1.0e+5, T_bar = T_bar)
s_arr_2,  tau_arr_2, w_arr_2, s_pi_arr_2, s_vis_arr_2, t_arr_1 = get_bond_slip(
    s_arr_1, dt,tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S,
    c=c, r=r, m=m, sigma_n=sigma_n, eta= eta * 1e-3, Cv=Cv, S0=1.0e+5, T_bar = T_bar)

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ax1 = plt.subplots(1,1)
fig.canvas.header_visible=False

ax1.plot(s_arr_1, tau_arr_1, '-k', linewidth=3, label="S0 = 0.25")
ax1.plot(s_arr_2, tau_arr_2, linewidth=3, color='green', label="S0 = 0.25")
# plt.title('stress-slip')
# plt.xlabel('Slip (mm)')
# plt.ylabel('Stress (MPa)')
# plt.legend(loc=4)